In [11]:
!pip install numpy
!pip install PyYAML


In [12]:
import numpy as np
import yaml
import random
import math

from typing import List, Callable

In [13]:
def load_config(filepath):
    with open(filepath, 'r') as f:
        return yaml.safe_load(f)

In [14]:
def generate_constants(constant_range, count):
    return [random.uniform(constant_range['min'], constant_range['max']) for _ in range(count)]

In [15]:
function_set = {
    '+': lambda a, b: a + b,
    '-': lambda a, b: a - b,
    '*': lambda a, b: a * b,
    '/': lambda a, b: a / b if b != 0 else 1.0  # Protected division
}

In [16]:
def generate_set(range_cfg):
    points = np.arange(range_cfg['min'], range_cfg['max'], range_cfg['step'])
    return points

In [18]:
class GPNode:
    def __init__(self, value, left=None, right=None):
        self.value = value  # operator or terminal
        self.left = left
        self.right = right

    def is_terminal(self):
        return self.left is None and self.right is None

    def evaluate(self, variables):
        if self.value in function_set:
            a = self.left.evaluate(variables)
            b = self.right.evaluate(variables)
            return function_set[self.value](a, b)
        elif self.value in variables:
            return variables[self.value]
        else:
            return self.value


In [19]:
def random_tree(functions, terminals, max_depth):
    if max_depth == 0 or (max_depth > 1 and random.random() < 0.5):
        value = random.choice(terminals)
        return GPNode(value)
    else:
        value = random.choice(functions)
        left = random_tree(functions, terminals, max_depth-1)
        right = random_tree(functions, terminals, max_depth-1)
        return GPNode(value, left, right)

In [20]:
def fitness(ind, x_points, y_points, target_values):
    errors = []
    for x, y, target in zip(x_points, y_points, target_values):
        try:
            val = ind.evaluate({'x': x, 'y': y})
        except Exception:
            val = float('inf')
        errors.append((val - target) ** 2)
    return np.mean(errors)

In [21]:
def genetic_programming(config):
    train_x = generate_set(config['train_range'])
    test_x = generate_set(config['test_range'])

    # If 2D problem, generate y accordingly (see your table, e.g. y = x + offset)
    train_y = train_x + 0.03 if 'y' in config['terminal_set'] else np.zeros_like(train_x)
    test_y = test_x + 0.03

    constants = generate_constants(config['constant_range'], config['constant_range']['count'])
    terminals = config['terminal_set'] + constants

    # Generate random individuals (trees)
    population = [random_tree(config['function_set'], terminals, max_depth=4) for _ in range(50)]

    # Evaluate fitness (assuming symbolic "problem" as ground truth)
    # You could use eval() or sympy for real problem function
    ground_truth = lambda x, y: eval(config['problem'], {"x": x, "y": y, "math": math, "exp": math.exp, "sin": math.sin, "cos": math.cos})
    target_values = [ground_truth(x, y) for x, y in zip(train_x, train_y)]

    for ind in population:
        fit = fitness(ind, train_x, train_y, target_values)
        print('Individual fitness:', fit)


In [22]:
config = load_config('config.yaml')
genetic_programming(config)

Individual fitness: 0.0035794644969419716
Individual fitness: 11.515914793142842
Individual fitness: 0.28738587217433
Individual fitness: 20.139340160486835
Individual fitness: 2480.699903244858
Individual fitness: 0.24302960284099903
Individual fitness: 0.07073180194429815
Individual fitness: 14.836954714199813
Individual fitness: 326.2934883521986
Individual fitness: 0.19825489635772142
Individual fitness: 14.502177001953473
Individual fitness: 28.21311553371392
Individual fitness: 0.6739891312353709
Individual fitness: 1.0488346830491468
Individual fitness: 0.17174010301243706
Individual fitness: 11.970422502524903
Individual fitness: 5.379736778913861
Individual fitness: 1.260895963609761
Individual fitness: 148.9184589787618
Individual fitness: 0.4214031802172019
Individual fitness: 1.022322992272046
Individual fitness: 0.42368444392052806
Individual fitness: 9.440028335762044
Individual fitness: 966.4574446998303
Individual fitness: 23.65083297170255
Individual fitness: 1.2193432